## 1. Introducción y motivación

### 1.1. Dataset elegido

Dataset de Uso de taxis Yellow Cab en USA en el año 2020.
https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [3]:
## IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

Tomamos los registros de los viajes de los Yellow Taxis en Nueva York de los meses del verano (julio, agosto y septiembre del año 2020), dado que el total de viajes del 2020 superaba los 24.000.000 de registros y no se podia trabajar con la libreria pandas.

In [5]:
# Ruta del archivo Parquet
file_path = 'dataset/dataset.parquet'

# Leer el archivo usando Pandas
df = pd.read_parquet(file_path)

df.shape

(3148715, 19)

## 2. Análisis exploratorio inicial

### - Visualizar las primeras filas.

In [ ]:
df.head()

### - Realizar un resumen de 5 números.

In [ ]:
df.describe().loc[['min', '25%', '50%', '75%', 'max']]

### - Identificar los tipos de datos: categórico, ordinal, etc. Responder para cada variable su tipo y si es informativa para un problema de clasificación (por ejemplo si se trata de un código, como una matrícula, o un nombre propio).

In [ ]:
# Identificar tipos de datos
data_types = df.dtypes

# Clasificación de tipos de datos
type_classification = {}
for column in df.columns:
    dtype = data_types[column]
    if pd.api.types.is_string_dtype(dtype):
        type_classification[column] = 'Categórico/Textual'
    elif pd.api.types.is_numeric_dtype(dtype):
        type_classification[column] = 'Numérico'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        type_classification[column] = 'Temporal'
    else:
        type_classification[column] = 'Otro'

# Evaluación de la informatividad para clasificación
informative_columns = {}
for column in df.columns:
    if type_classification[column] in ['Categórico/Textual', 'Numérico', 'Temporal']:
        # Heurística: los códigos únicos y nombres propios no son informativos
        if column.lower() in ['id', 'name', 'matricula', 'codigo','airport_fee','store_and_fwd_flag']:
            informative_columns[column] = 'No Informativa'
        else:
            informative_columns[column] = 'Informativa'
    else:
        informative_columns[column] = 'No Informativa'

# Crear un dataframe con la clasificación y evaluación
result_df = pd.DataFrame({
    'Tipo de Dato': [type_classification[col] for col in df.columns],
    'Informativa para Clasificación': [informative_columns[col] for col in df.columns]
}, index=df.columns)


# Mostrar el dataframe resultante
result_df

In [ ]:
# Usando pandas profiling
report = ProfileReport(df, title='Yellow Taxis in NY', minimal=True)
report

Como podemos ver, casi todas las variables son informativas debido a la naturaleza intrinseca del dataset, ya que la descripcion de cada variable implica y explica una relacion directa con la variable de salida seleccionada. Más adelante veremos la relacion directa entre cada variable de entrada y la variable de salida.

- store_and_fwd_flag: No es informativa debido a que no aporta ningun tipo de criterio util segun la descripcion de la variable, ademas de tener muchos valores nulos.
- airport_fee: No es informativa debido a que solo hay un solo valor para esta variable en todo el dataset.

### - Identificar las variables de entrada y de salida del problema.

Todas menos total_amount son variables de entrada. 

- Variables de entrada:
  - Realizar los siguientes análisis por tipo de variable:
    - **Numéricas:** Obtener conclusiones acerca de la distribución de los datos.
    - **Categorías:** Obtener conclusiones acerca de la cardinalidad, representación de cada categoría, etc.
    - **Compuestas:** ¿Pueden tratarse para utilizarse en el problema a resolver?

In [ ]:
features_df = df.drop(columns=['total_amount'])
features_df.hist(bins=30,log=False,figsize=(15,15))
plt.suptitle('Histogramas de Atributos y Target')
plt.show()

### Variables de salida (en caso de aplicar): 
o ¿Están balanceadas las clases? 
o (en caso de aplicar) ¿Qué técnicas consideraría para codificar la variable de salida? 
Justifique.

In [ ]:
# Ajustar las opciones de visualización para mostrar los números en formato estándar
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Resumen estadístico
describe_stats = df['total_amount'].describe()
print(describe_stats)

sns.histplot(df['total_amount'], kde=False, bins=90,log_scale=True)
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.title('Histograma con Seaborn')
plt.show()

# Crear un gráfico de caja
sns.boxplot(data=df, y='total_amount', log_scale=True)
plt.title('Gráfico de Caja')
plt.show()

## 3. Limpieza y preparación de datos / ingeniería de features
- Antes de entrenar un modelo de aprendizaje automático, ¿podría identificar las variables de entrada de mayor importancia? Considerar por lo menos dos técnicas para cada variable. Explique brevemente los métodos utilizados.

### - Datos faltantes. Indicar cantidad de observaciones y valores faltantes para cada variable.

In [ ]:
df.isnull().sum()

### - ¿Qué supuestos puede realizar acerca de los datos faltantes? ¿Qué técnicas de imputación recomendaría? Ensayar distintas técnicas y analizar los resultados.

#### - a. Eliminacion de columnas: Eliminaremos columnas completas que tienen un alto porcentaje de datos faltantes. 

En este caso, airport_fee tiene 2357493 datos faltantes.

In [ ]:
#Eliminar columna
df.drop(columns=['airport_fee'], inplace=True)

#Validar nuevo dataset sin la columna 'airport_fee'              
df.info()

#### - b. Eliminacion de datos faltantes: Eliminaremos las observaciones que contengan valores nulos en 'passenger_count', 'RatecodeID', 'congestion_surcharge' y 'store_and_fwd_flag'. Esto porque los valores no aportan informacion relevante a nuestra variable de salida y representan un numero reducido de muestras en la cuenta total de observaciones.

In [ ]:
#Eliminar valores nulos
df.dropna(inplace=True)
#Validar
df.isnull().sum()

### - En función del estudio inicial de las variables que se hizo en la sección anterior, elegir una técnica de codificación para cada variable. Cuando lo considere apropiado, ensayar distintas técnicas y comparar los resultados, teniendo en cuenta el tipo de clasificador a utilizar. Nota: para tipos de datos compuestos o estructurados, considerar la obtención de variables de tipo numérico/categórico.

> Vamos a implementar Binary Encoding en unica variable categorica que tenemos para reemplazar los valores de Y y N en 'store_and_fwd_flag':

In [ ]:
# Mostrar el DataFrame original
print("Variable original:")
print(df['store_and_fwd_flag'].head())

# Convertir 'Y' a 1 y 'N' a 0 usando map
df['store_and_fwd_flag'] = df['store_and_fwd_flag'].map({'Y': 1, 'N': 0})

# Mostrar el DataFrame convertido
print("Variable después de la conversión:")
print(df['store_and_fwd_flag'].head())

#### Chequeo de variables númericas:

> Primero vamos a ver si hay filas duplicadas y luego  vamos a chequear los limites de las columnas que tengan valores numéricos.  
Ninguna columna numérica debería tener valores negativos.

In [ ]:
#Para ver filas duplicadas
df[df.duplicated(keep=False)]

In [ ]:
size_antes = len(df)
df = df.drop_duplicates()
size_despues = len(df)
print(f'se eliminaron: {size_antes-size_despues} filas duplicadas')

In [ ]:
# se resetea
df.reset_index(drop=True, inplace=True)

In [ ]:
columnas_con_numeros = ['VendorID', 'passenger_count' ,'trip_distance','RatecodeID','PULocationID','DOLocationID','payment_type','fare_amount','extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge','total_amount','congestion_surcharge']
(df[columnas_con_numeros] < 0).any().to_frame('Menor que 0 ?')

In [ ]:
columnas_con_negativos = ['fare_amount','extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge','total_amount','congestion_surcharge']

In [ ]:
for c in columnas_con_negativos:
    print(c)
    display(df[df[c] < 0][c].value_counts().to_frame())
    print()

#### Eliminamos valores negativos y validamos las columnas 

In [ ]:
df_filtered = df[(df[columnas_con_negativos] >= 0).all(axis=1)]
(df_filtered[columnas_con_negativos] < 0).any().to_frame('Menor que 0 ?')


In [ ]:
df=df_filtered
df.reset_index(drop=True, inplace=True)

In [ ]:
df.describe().loc[['min', '25%', '50%', '75%', 'max']]

#### Outliers

In [ ]:
# Cálculo del IQR para la columna 'total_amount'
Q1 = df['total_amount'].quantile(0.25)
Q3 = df['total_amount'].quantile(0.75)
IQR = Q3 - Q1

# Define los límites inferior y superior para 'total_amount'
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtra el DataFrame para mantener solo las filas donde 'total_amount' esté dentro de los límites
df_sin_outliers = df[(df['total_amount'] >= lower_bound) & (df['total_amount'] <= upper_bound)]

# Imprimir el número de filas eliminadas
print(f"Número de filas eliminadas: {len(df) - len(df_sin_outliers)}")

In [ ]:
df_sin_outliers.reset_index(drop=True, inplace=True)

In [ ]:
df_sin_outliers.describe().loc[['min', '25%', '50%', '75%', 'max']]

In [ ]:
df_sin_outliers['passenger_count'].value_counts()

In [ ]:
df = df_sin_outliers

In [ ]:
df.describe()

### - ¿Qué puede decir acerca de las relaciones entre las variables de entrada?

In [ ]:
correlation_matrix = df.corr()
plt.figure(figsize=(15, 15))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')

plt.title('Heatmap de Correlaciones')
plt.show()

In [ ]:
# Resumen estadístico
describe_stats = df['total_amount'].describe()
print(describe_stats)

sns.histplot(df['total_amount'], kde=False, bins=90,log_scale=False)
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.title('Histograma con Seaborn')
plt.show()

# Crear un gráfico de caja
sns.boxplot(data=df, y='total_amount', log_scale=False)
plt.title('Gráfico de Caja')
plt.show()

## Para elegir las variables de más importancia, vamos a  usar la técnica de filtrado de variables de forma estadística y técnicas embedded por árbol de decisión.

### Por varianza, se define un umbral mínimo para considerar variables. Por defecto elimina las features de varianza 0 (sin cambios) <br>
Para asegurarnos que funcione agreguemos variables con esas condiciones

In [ ]:
_df = df.copy()
_df['with_zero_variance'] = 10
_df['with_low_variance'] = np.random.uniform(0, 0.2, _df.shape[0])

In [ ]:
_df.head()

In [ ]:
_df.select_dtypes(include=['number']).var()

In [ ]:


def filter_by_variance(df, threshold):
    # Columnas con varianza calculable
    cols_con_varianza = df.var().index.values
    _df = df[cols_con_varianza].copy()
    print(f'columnas antes: {_df.columns.tolist()}')

    # calculo varianzas
    selector = VarianceThreshold(threshold=threshold)
    vt = selector.fit(_df)

    ## vt.get_support() me da los indices de las columnas que quedaron
    _df = _df.loc[:, vt.get_support()]
    print(f'columnas que quedan: {_df.columns.tolist()}')


filter_by_variance(_df.select_dtypes(include=['number']), 0)
print()
filter_by_variance(_df.select_dtypes(include=['number']), 10)

### Recursive Feature Eliminator a través de Arboles de Decisión con Regresión Lineal porque las variables son continuas.
 



In [ ]:

columnas_con_numeros = ['VendorID', 'passenger_count' ,'trip_distance','RatecodeID','PULocationID','DOLocationID','payment_type','fare_amount','extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge',     'total_amount','congestion_surcharge']


# Definir X e y. Supongamos que la etiqueta está en una columna llamada 'total_amount'
X = df[columnas_con_numeros].drop(columns=['total_amount'])  # Elimina la columna de la etiqueta de X
y = df[columnas_con_numeros]['total_amount']  # La columna de la etiqueta

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



In [ ]:
# Entrenar el modelo Decision Tree (como las variables son continuas usando regresión)

# Entrenar el modelo Decision Tree Regressor
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)



In [ ]:
#Predecir 
y_pred = model.predict(X_test)

In [ ]:
#Calcular métricas de evaluación para regresión

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'R^2 Score: {r2:.2f}')    

In [ ]:
# Obtener la importancia de las características
importances = model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})

# Mostrar la importancia de las características
print("Importancia de las características:")
print(feature_importance_df)

Preguntas interesantes para considerar aquí: (elija una o dos)

• ¿Existe una manera de caracterizar los lugares más recurrentes para
inicio/fin de viaje? 

- Habría que reemplazar PULocationID y DOLocationID por su respectivo valor zonal, usando la columna "Zone" de 'dataset\taxi_zone_lookup.csv', y ver qué combinaciones son las mas frecuentes.
    

• ¿Cómo son los viajes típicamente en distancia y tiempo?

- Para la Distancia: usar trip_distance
- Para el Tiempo:Crear una nueva columna restanto tpep_dropoff_datetime y tpep_pickup_datetime, de aqui tenemos el tiempo total de viaje 

- IDEA: En que franja horaria ocurre la mayor cantidad de viajes?



• ¿Podremos segmentar los viajes de alguna manera? (clusterización)

- Ni idea, capaz por Mes y ver qué mes tiene mas viajes, cuales son más caros, etc.